In [1]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
import nltk

In [2]:
URI = 'http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter'
# URI = 'http://dbkwik.webdatacommons.org/Game_of_Thrones_Wik/resource/Daenerys_Targaryen'

In [3]:
def get_basic_info(URI):
    wiki = URI.split('/')[3]
    ontology_namespace = "http://dbkwik.webdatacommons.org/" + wiki + "/ontology"
    property_namespace = "http://dbkwik.webdatacommons.org/" + wiki + "/property"
    
    sparql = SPARQLWrapper("http://dbkwik.webdatacommons.org/sparql")
    query = ("""SELECT (group_concat(?type;separator='|') as ?types) ?name ?gender ?dbr WHERE {        
        # Get Types of URI
        <""" + URI + """> rdf:type ?type .
        FILTER(contains(str(?type), '""" + ontology_namespace + """')) .
        
        # Get English label of URI
        OPTIONAL { <""" + URI + """> <""" + property_namespace + """/name> ?name . FILTER(lang(?name)='en') . }
        OPTIONAL { <""" + URI + """> <http://www.w3.org/2004/02/skos/core#prefLabel> ?name . FILTER(lang(?name)='en') . }
                
        # Try to get gender
        OPTIONAL { <""" + URI + """> <""" + property_namespace + """/gender> ?gender . }
        
        # Try to get corresponding DBpedia Resource
        OPTIONAL { <""" + URI + """> owl:sameAs ?dbr . }
    }
    """)
    print(query)
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    output = {}
    for result in results["results"]["bindings"]:
        print(result)
        output = {
            'types': result['types']['value'],            
            'dbr': result['dbr']['value']
        }
        
        if 'name'in result:
            output['name'] = result['name']['value']
        if 'gender' in result:
            output['gender'] = result['gender']['value']
        break
        
    return output


get_basic_info(URI)

SELECT (group_concat(?type;separator='|') as ?types) ?name ?gender ?dbr WHERE {        
        # Get Types of URI
        <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> rdf:type ?type .
        FILTER(contains(str(?type), 'http://dbkwik.webdatacommons.org/HarryPotter/ontology')) .
        
        # Get English label of URI
        OPTIONAL { <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> <http://dbkwik.webdatacommons.org/HarryPotter/property/name> ?name . FILTER(lang(?name)='en') . }
        OPTIONAL { <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> <http://www.w3.org/2004/02/skos/core#prefLabel> ?name . FILTER(lang(?name)='en') . }
                
        # Try to get gender
        OPTIONAL { <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> <http://dbkwik.webdatacommons.org/HarryPotter/property/gender> ?gender . }
        
        # Try to get corresponding DBpedia Resource
        OPTIONAL { <htt

{'dbr': 'http://dbpedia.org/resource/Harry_Potter',
 'gender': 'Male',
 'name': 'Harry James Potter',
 'types': 'http://dbkwik.webdatacommons.org/HarryPotter/ontology/Actor|http://dbkwik.webdatacommons.org/HarryPotter/ontology/Agent|http://dbkwik.webdatacommons.org/HarryPotter/ontology/Artist|http://dbkwik.webdatacommons.org/HarryPotter/ontology/Person'}

In [6]:
def get_top_k_triples(URI, k):
    wiki = URI.split('/')[3]
    ontology_namespace = "http://dbkwik.webdatacommons.org/" + wiki + "/ontology"
    property_namespace = "http://dbkwik.webdatacommons.org/" + wiki + "/property"
    
    sparql = SPARQLWrapper("http://ec2-18-219-186-206.us-east-2.compute.amazonaws.com:3030/dbkwik/query")
    query = ("""SELECT ?predicate (group_concat(distinct ?resource; separator="|") as ?resources) (AVG(?rank) as ?ranks)
        WHERE {
          {
            # URI - Predicate - Resource
            select ?predicate ?resource ?rank {
            <""" + URI + """> ?predicate ?resource .
            ?resource <http://purl.org/voc/vrank#pagerank> ?rank .
            }
          }
          UNION
          {
            # Resource - Predicate - URI
            select ?predicate ?resource ?rank {
            ?resource ?predicate <""" + URI + """> .
            ?resource <http://purl.org/voc/vrank#pagerank> ?rank .
            }
          }

          FILTER (?predicate NOT IN (<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, 
                <http://purl.org/dc/terms/subject>, 
                <http://xmlns.com/foaf/0.1/depiction>, 
                <http://www.w3.org/2002/07/owl#sameAs>, 
                <""" + ontology_namespace + """/thumbnail>, 
                <""" + property_namespace + """/predecessor>,
                <""" + property_namespace + """/successor>, 
                <http://xmlns.com/foaf/0.1/isPrimaryTopicOf>, 
                <http://xmlns.com/foaf/0.1/primaryTopic>)).
        } GROUP BY ?predicate ?ranks ORDER BY DESC(?ranks)
        LIMIT """ + str(k) + """
    """)
    
    print(query)
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    output = []
    for result in results["results"]["bindings"]:
        obj = {
            'predicate': result['predicate']['value'],
            'resources': result['resources']['value'],
            'ranks': result['ranks']['value']
        }
        
        output.append(obj)        
    return output

get_top_k_triples(URI, 10)

SELECT ?predicate (group_concat(distinct ?resource; separator="|") as ?resources) (AVG(?rank) as ?ranks)
        WHERE {
          {
            # URI - Predicate - Resource
            select ?predicate ?resource ?rank {
            <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> ?predicate ?resource .
            ?resource <http://purl.org/voc/vrank#pagerank> ?rank .
            }
          }
          UNION
          {
            # Resource - Predicate - URI
            select ?predicate ?resource ?rank {
            ?resource ?predicate <http://dbkwik.webdatacommons.org/HarryPotter/resource/Harry_Potter> .
            ?resource <http://purl.org/voc/vrank#pagerank> ?rank .
            }
          }

          FILTER (?predicate NOT IN (<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, 
                <http://purl.org/dc/terms/subject>, 
                <http://xmlns.com/foaf/0.1/depiction>, 
                <http://www.w3.org/2002/07/owl#sameAs>, 
           

[{'predicate': 'http://dbkwik.webdatacommons.org/HarryPotter/property/species',
  'ranks': '39.11966390386006e0',
  'resources': 'http://dbkwik.webdatacommons.org/HarryPotter/resource/Human'},
 {'predicate': 'http://dbkwik.webdatacommons.org/HarryPotter/property/born',
  'ranks': '9.022693502101427e0',
  'resources': "http://dbkwik.webdatacommons.org/HarryPotter/resource/England|http://dbkwik.webdatacommons.org/HarryPotter/resource/Great_Britain|http://dbkwik.webdatacommons.org/HarryPotter/resource/Godric's_Hollow|http://dbkwik.webdatacommons.org/HarryPotter/resource/West_Country"},
 {'predicate': 'http://dbkwik.webdatacommons.org/HarryPotter/property/house',
  'ranks': '4.791282557190195e0',
  'resources': 'http://dbkwik.webdatacommons.org/HarryPotter/resource/Gryffindor'},
 {'predicate': 'http://dbkwik.webdatacommons.org/HarryPotter/property/blood',
  'ranks': '0.9664948751490945e0',
  'resources': 'http://dbkwik.webdatacommons.org/HarryPotter/resource/Half-blood'},
 {'predicate': 'h